根据附件 1 景区及酒店网评文本及附件 2 景区及酒店得分建立合理的数学模型及相应
算法，按满分为 5 分对景区及酒店的服务、位置、设施、卫生、性价比五个方面进行评分，
并按照均方误差（Mean Squared Error, MSE）进行模型评价。

评价模型的训练，感觉其实是一个监督学习中的回归问题。但这个机器学习，数据集有些少，就 50 个。

如何解决呢？首先采用分词，再用词袋模型将非结构化的文本，转换为结构化的向量。然后，将对同一个地点的评价，的结构化向量拼接成一个矩阵，然后就可以用计算机视觉的方法解决了。

In [24]:
import pandas as pd
import numpy as np
import jieba
from sklearn.externals import joblib
#定义停用词、标点符号
punctuation = ["，","。","","；", "？"]
#定义语料
jqcomment = pd.read_excel('D:/泰迪杯/c题/附件1/景区评论.xlsx')
jqscore = pd.read_excel('D:/泰迪杯/c题/附件2/景区评分.xlsx')

In [25]:
jqcomment.index=jqcomment['景区名称']

In [26]:
z1 = [f'A{x}{y}' for x in range(0,5) for y in range(0,10)][1:]
df1 = [jqcomment.loc[f'{x}']['评论内容'] for x in z1]

In [27]:
df2 = [jqcomment.loc['A50']['评论内容']]

In [28]:
x = df1+df2
x=[str(i) for i in x]
x

['景区名称\nA01    是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...\nA01    **景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...\nA01    很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...\nA01    有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是34...\nA01    周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票198 需要出示相关证件（校卡或...\n                             ...                        \nA01    我个人非常喜欢这种刺激休闲的地方 只不过放假人非常多 所以排队什么的都要很久 下次还会再去的...\nA01    很不错哦 对比了三个APP的价格什么的，最后选择了去哪儿。。棒棒哒。碰到昨天天气不冷不热 玩...\nA01                                        就是动物园不好玩 不值得去\nA01    Feb. 9 Day3：乘轻轨从珠海前往广州南（我真的不太能分清轻轨、高铁和动车），开启第二...\nA01    游**动物园周末，我和妈妈怀着兴奋的心情来到亚洲最大的**景区——**动物园游玩。一进门，我...\nName: 评论内容, Length: 4663, dtype: object',
 '景区名称\nA02    第一次来海洋馆，感觉好神奇啊！水母，白鲸，海豚，海豹，海象，企鹅，北极狐，北极熊，各有各的可...\nA02                           值得一去的游玩场所，不管小孩，不同年龄玩的项目都有。\nA02    珠海横琴**景区一日游小结：前提，是实在不想辜负长长假期，小宝每天把珠海**挂嘴边，所以就网...\nA02    少一颗星是因为88元是抵消卷，不是管一份，里面的餐厅很多不开，我建议游客带面包进去，餐厅的东...\nA02    比想象中的要好玩儿太多，绝对值得这个票价。几大剧场一定要去看，5D电影体验感超级强，让人身临...\n              

In [29]:
import re
#加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='gb2312').readlines()]
# print(stoplist)
sentences_list = x
# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)

x = sentences_list

In [30]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfVectorizer()

x1 = tf_idf_transformer.fit_transform(x)
type(x1)

scipy.sparse.csr.csr_matrix

In [31]:
x1

<50x1986 sparse matrix of type '<class 'numpy.float64'>'
	with 2404 stored elements in Compressed Sparse Row format>

In [35]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV

In [58]:
dt = DecisionTreeClassifier(max_depth = None)
bc = BaggingClassifier(dt, n_estimators = 10, max_samples = 1.0, max_features = 1.0)

In [59]:
clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
                                     ),
                   param_grid)
clf.fit(x_train,y_train.astype('str'))

D:\Anacnda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
D:\Anacnda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
D:\Anacnda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                              

In [42]:
print(y_test)

15    3.9
29    3.9
9     3.8
48    3.9
17    3.9
30    3.9
34    4.0
32    3.9
40    4.0
42    3.9
Name: 服务得分, dtype: float64


In [34]:
#服务得分评分
y = jqscore
y1 = y['服务得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y1,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
joblib.dump(model_BaggingRegressor1,'B1.model')
model_BaggingRegressor1.fit(x_train,y_train)
y_hat = model_BaggingRegressor1.predict(x_test)
print("服务得分评分：",y_hat.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat,y_test))

服务得分评分： 3.8899999999999992
代价函数MSE： 0.02008


In [117]:
#位置得分评分
y = jqscore
y2 = y['位置得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y2,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor2 = BaggingRegressor()
model_BaggingRegressor2.fit(x_train,y_train)
y_hat1 = model_BaggingRegressor2.predict(x_test)
joblib.dump(model_BaggingRegressor2,'B2.model')
print("服务得分评分：",y_hat1.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat1,y_test))

服务得分评分： 4.630000000000001
代价函数MSE： 0.007289999999999977


In [118]:
#设施得分评分
y = jqscore
y3 = y['设施得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y3,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor3 = BaggingRegressor()
model_BaggingRegressor3.fit(x_train,y_train)
y_hat2 = model_BaggingRegressor3.predict(x_test)
joblib.dump(model_BaggingRegressor3,'B3.model')
print("服务得分评分：",y_hat2.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat2,y_test))

服务得分评分： 4.2299999999999995
代价函数MSE： 0.2500500000000002


In [119]:
#设施得分评分
y = jqscore
y4 = y['卫生得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y4,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor4 = BaggingRegressor()
model_BaggingRegressor4.fit(x_train,y_train)
y_hat3 = model_BaggingRegressor4.predict(x_test)
joblib.dump(model_BaggingRegressor4,'B4.model')
print("服务得分评分：",y_hat3.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat3,y_test))

服务得分评分： 4.8199999999999985
代价函数MSE： 0.09923999999999994


In [120]:
#设性价比得分评分
y = jqscore
y5 = y['性价比得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y5,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor5 = BaggingRegressor()
model_BaggingRegressor5.fit(x_train,y_train)
y_hat4 = model_BaggingRegressor5.predict(x_test)
joblib.dump(model_BaggingRegressor5,'B5.model')
print("服务得分评分：",y_hat4.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat4,y_test))

服务得分评分： 4.3100000000000005
代价函数MSE： 0.055259999999999886


In [121]:
y = jqscore
y6 = y['总得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y6,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor6 = BaggingRegressor()
model_BaggingRegressor6.fit(x_train,y_train)
y_hat5 = model_BaggingRegressor6.predict(x_test)
joblib.dump(model_BaggingRegressor6,'B6.model')
print("服务得分评分：",y_hat5.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat5,y_test))

服务得分评分： 4.336
代价函数MSE： 0.034547600000000005


In [12]:
BaggingRegressor?